The goal of this assignment is to read in a large dataset of protein annotation information and to manipulate, summarize and analyze it using Dask Dataframes.

Protein annotation is a branch of bioinformatics which classifies the different parts of a protein's structure based on both sequence and functional characteristics. For instance, it recognizes structural elements like trans-membrane helices, but also particular active sites ("Serine Protease") and also signal peptides ("periplasmic membrane tag"). The holy grail of this field is to use these different annotations of parts of the protein sequence, and to combine them to predict the function of the protein as a whole, without having to carry out actual experiments in the lab.

The subject is the output of the InterProScan protein annotation service InterproScan online, NAR article. Briefly, InterPROscan is a meta-annotator: it runs different protein function annotators in turn on an input amino-acid sequence FASTA file and collects the output of each, labelling them with a unique and consistent identifier – the "InterPRO number". This service is used to annotate all currently known prokaryotic (Bacteria, Archaea) genomes to investigate better methods of metagenomics sequence annotation.

An explanation of the data can be found here

2. Deliverables¶

Write a script that reads in a InterPROscan output file and answers the questions below. You can test your script on the data-file that you can find at /data/dataprocessing/interproscan/all_bacilli.tsv file on assemblix2012 and assemblix2019. You must use the Dask Dataframe interface to read in and manipulate this file. This file contains ~4,200,000 protein annotations and is around 11GB.

How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?

How many annotations does a protein have on average?

What is the most common GO Term found?

What is the average size of an InterPRO feature found in the dataset?

What is the top 10 most common InterPRO features?

If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?

If you look at those features which also have textual annotation, what is the top 10 most common word found in that annotation?

And the top 10 least common?

Combining your answers for Q6 and Q7, what are the 10 most commons words found for the largest InterPRO features?

What is the coefficient of correlation () between the size of the protein and the number of features found?

Notes: 

- InterPRO annotations are accession numbers, e.g. IPRO002093) 

- If a value is missing, a - is displayed. 

- One cell can contain multiple GO annotations divided by |

NB1: Make sure you use the /commons/conda environment

NB2: Do not download the file on your local machine, but work on it from the server. You can configure VSCode on your machine to connect (via ssh) to assemblix2019.

NB3: Use only 16 threads maximum: dask.dataframe.compute(num_workers=16)

In [21]:
import dask.dataframe as ddf
ddf.compute(num_workers=16)


()

In [22]:
header = ['protein_accession', 'MD5', 'length', 'analysis', 
          'signature_accession', 'signature_description', 'start_loc', 
          'stop_loc', 'score', 'status', 'date', 'ip_annot_accession',
          'ip_annot_desc',"GO annotations","Pathways annotations"]

dtypes = {'protein_accession':str, 'MD5':str, 'length':int, 'analysis':str, 
          'signature_accession':str, 'signature_description':str, 'start_loc':int, 
          'stop_loc':int, 'score':str, 'status':str, 'date':str, 'ip_annot_accession':str,
          'ip_annot_desc':str}

ipro_ddf = ddf.read_table("/data/dataprocessing/interproscan/all_bacilli.tsv",header=None, names=header, dtype=dtypes, delimiter="\t", quoting=3)


ipro_ddf.head()

,protein_accession,MD5,length,analysis,signature_accession,signature_description,start_loc,stop_loc,score,status,date,ip_annot_accession,ip_annot_desc,GO annotations,Pathways annotations
0,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03882,cyclo_dehyd_2: bacteriocin biosynthesis cyclod...,2,131,1.6E-21,T,25-04-2022,IPR022291,"Bacteriocin biosynthesis, cyclodehydratase domain",-,-
1,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03604,TOMM_cyclo_SagD: thiazole/oxazole-forming pept...,161,547,0.0,T,25-04-2022,IPR027624,"Thiazole/oxazole-forming peptide maturase, Sag...",-,-
2,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,ProSiteProfiles,PS51664,YcaO domain profile.,159,547,75.396477,T,25-04-2022,IPR003776,YcaO-like domain,-,-
3,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Gene3D,G3DSA:3.30.160.660,-,311,452,9.1E-46,T,25-04-2022,-,-,NaN,NaN
4,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Gene3D,G3DSA:3.30.40.250,-,191,274,9.1E-46,T,25-04-2022,-,-,NaN,NaN


In [23]:
# 1. How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?
# 2. How many annotations does a protein have on average?
ip_annot_num = ipro_ddf["ip_annot_accession"].nunique().compute()
prot_annot_num = ipro_ddf["protein_accession"].nunique().compute()
print(f'Unique interPRO numbers: {ip_annot_num}\n'
      f'Unique protein numbers: {prot_annot_num}\n'
      f'Amount of annotations per protein on average: {prot_annot_num//ip_annot_num}')

Unique interPRO numbers: 9704
Unique protein numbers: 365570
Amount of annotations per protein on average: 37


In [24]:
# 3. What is the most common GO Term found?
val_counts = ipro_ddf["GO annotations"].value_counts().compute()
val_counts = val_counts[1:]
separated_val_counts = {key:0 for key in [key for keys in list(val_counts.keys()) for key in keys.split('|')]}

for keys, value in val_counts.items():
    for key in keys.split('|'):
        separated_val_counts[key] += value

print(f'Most common GO term: {max(separated_val_counts,key=separated_val_counts.get)}')

Most common GO term: GO:0005524


In [25]:
# 4. What is the average size of an InterPRO feature found in the dataset?
ip_size = ipro_ddf.groupby('ip_annot_accession').mean().compute()
ip_size["feature_length"] = ip_size["stop_loc"] - ip_size["start_loc"]
ip_size.drop('-', inplace=True)
print(f"average size of an InterPRO feature: {ip_size['feature_length'].mean():.05}")




average size of an InterPRO feature: 184.33


In [26]:
# 5. What is the top 10 most common InterPRO features?
ip_counts = ipro_ddf["ip_annot_accession"].value_counts().compute()

print(f'Top 10 most common IP features:\n{ip_counts[1:11]}')

Top 10 most common IP features:
IPR027417    46834
IPR002347    18077
IPR003439    16944
IPR036388    16291
IPR036259    12602
IPR003593    11521
IPR036390    11304
IPR036291    10716
IPR000515    10613
IPR001789    10471
Name: ip_annot_accession, dtype: int64


In [27]:
# 6. If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then.

ip_size = ip_size[ip_size["feature_length"] > (ip_size["length"]*.9)].reset_index()


filt_ip_counts = ip_counts[ip_counts.index.isin(ip_size["ip_annot_accession"])]

print(f'Top 10 most common IP features > 90% protein length:\n{filt_ip_counts[1:11]}')

Top 10 most common IP features > 90% protein length:
IPR034660    2040
IPR039420    1968
IPR004761    1454
IPR029056    1409
IPR036812    1291
IPR000944    1261
IPR036380    1129
IPR017439    1124
IPR004995    1062
IPR003188    1059
Name: ip_annot_accession, dtype: int64


In [30]:
# 7. If you look at those features which also have textual annotation, what is the top 10 most common word found in that annotation?
word_counts = ipro_ddf['ip_annot_desc'].str.lower().replace(',', '').str.split().explode().value_counts().compute()


print(f'Top 10 most common words in IP annot description:\n{word_counts[1:11]}')


Top 10 most common words in IP annot description:
domain         636803
superfamily    322262
protein        215412
c-terminal     100810
site            74709
hydrolase       73452
dna-binding     64627
n-terminal      64427
type            58783
conserved       57523
Name: ip_annot_desc, dtype: int64


In [31]:
# 8. And the top 10 least common?

print(f'Top 10 least common words in IP annot description:\n{word_counts[:-11:-1]}')

Top 10 least common words in IP annot description:
chp04141            1
hygromycin-b        1
napa-like,          1
hyicolysin          1
deda                1
hypoxanthine-dna    1
hypx/hoxx           1
defence             1
deficient           1
i-b/tneap           1
Name: ip_annot_desc, dtype: int64


In [32]:
# 9. Combining your answers for Q6 and Q7, what are the 10 most commons words found for the largest InterPRO features?

filt_ip_words = ipro_ddf['ip_annot_desc'][ipro_ddf["ip_annot_accession"].isin(ip_size["ip_annot_accession"])].str.lower().replace(',', '').str.split().explode().value_counts().compute()

print(f'Top 10 most common words in IPs > 90% of protein, annot description:\n{filt_ip_words[1:11]}')

Top 10 most common words in IPs > 90% of protein, annot description:
superfamily    34027
domain         20306
subunit        19550
family         17339
ribosomal      15644
synthase       13459
type           10183
of              9784
unknown         9266
function        8986
Name: ip_annot_desc, dtype: int64


In [59]:
# Count the number of rows with the same 'Protein Accession'
count_df = ipro_ddf.groupby('protein_accession').size().reset_index().compute()

# Merge the count DataFrame with the original DataFrame on 'Protein Accession'
prot_count_length_df = ddf.merge(count_df, ipro_ddf[['protein_accession','length']], on='protein_accession').drop_duplicates('protein_accession').compute()

prot_count_length_df.rename(columns={0:'count'},inplace=True)

corr = prot_count_length_df['length'].corr(prot_count_length_df['count'])

print(f'The R squared correlation coefficient between protein length and number of features: {corr**2:.03}')



The R squared correlation coefficient between protein length and number of features: 0.408
